In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchinfo import summary

In [2]:
use_mps = torch.backends.mps.is_available()
use_cuda = torch.cuda.is_available()
device = "cpu"
if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")

# device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='mps')

In [3]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Some Notes on our naive model

We are going to write a network based on what we have learnt so far.

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast".

In [4]:
class FirstDNN(nn.Module):
  def __init__(self):
    super(FirstDNN, self).__init__()
    # r_in: 1, n_in: 28, j_in: 1, s: 1, r_out: 3, n_out: 28, j_out: 1
    self.conv1 = nn.Conv2d(1, 32, 3, padding=1)

    # r_in: 3, n_in: 28, j_in: 1, s: 1, r_out:5 , n_out: 28, j_out: 1
    self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

    # r_in: 5, n_in: 28, j_in: 1, s: 2, r_out: 6, n_out: 14, j_out: 2
    self.pool1 = nn.MaxPool2d(2, 2)

    # r_in: 6, n_in:14 , j_in: 2, s: 1, r_out: 10, n_out: 14, j_out: 2
    self.conv3 = nn.Conv2d(64, 128, 3, padding=1)

    # r_in: 10, n_in: 14, j_in: 2, s: 1, r_out: 14, n_out: 14, j_out: 2
    self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)

    # r_in: 14, n_in: 14, j_in: 2, s:2 , r_out: 16, n_out: 7, j_out: 4
    self.pool2 = nn.MaxPool2d(2, 2)

    # r_in: 16, n_in: 7, j_in: 4, s: 1, r_out: 24, n_out: 5, j_out: 4
    self.conv5 = nn.Conv2d(256, 512, 3)

    # r_in: 24, n_in: 5, j_in: 4, s: 1, r_out: 32, n_out: 3, j_out: 4
    self.conv6 = nn.Conv2d(512, 1024, 3)

    # r_in: 32, n_in: 3, j_in: 4, s: 1, r_out: 40, n_out: 1, j_out: 4
    self.conv7 = nn.Conv2d(1024, 10, 3)

# Correct values
# https://user-images.githubusercontent.com/498461/238034116-7db4cec0-7738-42df-8b67-afa971428d39.png
  def forward(self, x):
    x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
    x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
    x = F.relu(self.conv6(F.relu(self.conv5(x))))
    x = self.conv7(x)
    # x = F.relu(x) # this is the last step. Think what ReLU does to our results at this stage!
    x = x.view(-1, 10)
    return F.log_softmax(x)


In [5]:
model = FirstDNN().to(device)

In [6]:
summary(model, input_size=(1, 28, 28))

/var/folders/g5/3xfgn5ts5pb5p_7sdybcfdlc0000gn/T/ipykernel_5387/1406177917.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Layer (type:depth-idx)                   Output Shape              Param #
FirstDNN                                 [1, 10]                   --
├─Conv2d: 1-1                            [32, 28, 28]              320
├─Conv2d: 1-2                            [64, 28, 28]              18,496
├─MaxPool2d: 1-3                         [64, 14, 14]              --
├─Conv2d: 1-4                            [128, 14, 14]             73,856
├─Conv2d: 1-5                            [256, 14, 14]             295,168
├─MaxPool2d: 1-6                         [256, 7, 7]               --
├─Conv2d: 1-7                            [512, 5, 5]               1,180,160
├─Conv2d: 1-8                            [1024, 3, 3]              4,719,616
├─Conv2d: 1-9                            [10, 1, 1]                92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
Total mult-adds (G): 18.74
Input size (MB): 0.00
Forward/backward pass size (MB): 1.38
Params size (MB): 25.52
Estima

In [9]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    print(device)
    model.train()
    model.to(device)
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    model.to(device)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

mps


  0%|          | 0/469 [00:00<?, ?it/s]/var/folders/g5/3xfgn5ts5pb5p_7sdybcfdlc0000gn/T/ipykernel_5387/1406177917.py:40: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06411048769950867 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.15it/s] 



Test set: Average loss: 0.0667, Accuracy: 9786/10000 (98%)

